In [1]:
import os
import pandas as pd
import json

# IMDB
data: https://www.kaggle.com/datasets/ashirwadsangwan/imdb-dataset  \
saved name.basics as names, title.basics as titles, title.ratings as ratings  \
redownloaded title.basics from [kaggle](https://www.kaggle.com/datasets/ashirwadsangwan/imdb-dataset/versions/699?resource=download) \
* names columns = const, primaryName, birthYear, deathYear, primaryProfession, knownForTitles (array of tconst)   
* titles columns = tconst, titleType, primaryTitle, originalTitle, isAdult(1=adult title), startYear, endYear, runtimeMinutes, genres (string array, max length=3)
* ratings columns =  tconst, averageRating, numVotes

In [2]:
folder = './imdb/'

git bash: 
* head -n 1 titles.tsv > title_sample.tsv
* tail -n+2 titles.tsv | shuf -n 1000000 >> title_sample.tsv

In [3]:
files = [os.path.join(folder, file) for file in os.listdir(folder)]
files

['./imdb/names.tsv',
 './imdb/ratings.tsv',
 './imdb/titles.tsv',
 './imdb/title_sample.tsv']

In [ ]:
# dfs = {f"{os.path.basename(files[0]).split('.')[0]}_df": pd.read_csv(file, delimiter="\t") for file in files}
# DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.

In [4]:
ratings = pd.read_csv(files[1], delimiter='\t', dtype={'tconst': str, 
                                                       'averageRating': 'float64', 
                                                       'numVotes': int
                                                      }, usecols=['tconst', 'averageRating'])

In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238892 entries, 0 to 1238891
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1238892 non-null  object 
 1   averageRating  1238892 non-null  float64
dtypes: float64(1), object(1)
memory usage: 18.9+ MB


In [ ]:
# names = pd.read_csv(files[0], delimiter='\t', usecols=['primaryName', 'knownForTitles'])

In [ ]:
# names.info()

In [ ]:
# names.head()

In [ ]:
# sample
titles_sample = pd.read_csv(files[3], delimiter='\t')
titles_sample.genres.unique()

In [192]:
t_columns = {'tconst': str,
            'titleType': str,
            'primaryTitle': str,
            'originalTitle': str,
            'isAdult': object,
            'startYear': object,
            'endYear': object,
            'runtimeMinutes': object,
            'genres': object
            }

In [196]:
titles = pd.read_csv(files[3], delimiter='\t', dtype=t_columns, usecols=['tconst', 
                                                                         'titleType', 
                                                                         'primaryTitle',
                                                                         'isAdult', 
                                                                         'startYear', 
                                                                         'runtimeMinutes', 
                                                                         'genres'])

In [194]:
# isAdult
titles.isAdult.unique()

array(['0', '1', '2014'], dtype=object)

In [197]:
# titles.loc[titles.isAdult == '2014'] = tvEpisode

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres
149201,tt3984412,tvEpisode,"I'm Not Going to Come Last, I'm Just Going to ...",2014,\N,"Game-Show,Reality-TV",NaN


In [198]:
titles.drop(titles[titles.isAdult != '0'].index, inplace=True)

In [199]:
titles.drop(['isAdult'], axis=1, inplace=True)

In [31]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 969110 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          969110 non-null  object
 1   titleType       969110 non-null  object
 2   primaryTitle    969110 non-null  object
 3   startYear       969110 non-null  object
 4   runtimeMinutes  969110 non-null  object
 5   genres          969110 non-null  object
dtypes: object(6)
memory usage: 51.8+ MB


In [200]:
# titleType
titles.titleType.unique()

array(['tvEpisode', 'video', 'short', 'movie', 'tvSeries', 'tvMovie',
       'tvMiniSeries', 'videoGame', 'tvSpecial', 'tvShort'], dtype=object)

In [201]:
titles.drop(titles[titles.titleType != 'movie'].index, inplace=True)

In [235]:
titles.drop(['titleType'], axis=1, inplace=True)

In [52]:
def drop_n(df):
    for column in df.columns:
        df.drop(df[df[column] == '\\N'].index, inplace=True)

In [202]:
drop_n(titles)

In [203]:
# genre
titles.drop(titles[titles.genres.isnull()].index, inplace=True)

In [204]:
# year
titles.startYear = titles.startYear.astype('int64')

In [205]:
len(titles.loc[titles.startYear < 1942]) # 2610 rows

2610

In [206]:
titles.drop(titles.loc[titles.startYear < 1942].index, inplace=True)

In [207]:
# runtime
titles.runtimeMinutes = titles.runtimeMinutes.astype('int64')

In [208]:
len(titles[titles.runtimeMinutes < 50]) # 1071 rows

1071

In [209]:
# left join ratings
t_ratings = titles.merge(ratings, how='left', on='tconst')

In [210]:
t_ratings.isnull().sum()/(len(t_ratings)) # ratings 30% null

tconst            0.000000
titleType         0.000000
primaryTitle      0.000000
startYear         0.000000
runtimeMinutes    0.000000
genres            0.000000
averageRating     0.307527
dtype: float64

In [236]:
# inner join ratings
t_with_ratings = titles.merge(ratings, how='inner', on='tconst')

In [237]:
 t_with_ratings.info() # 25107 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25107 entries, 0 to 25106
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          25107 non-null  object 
 1   primaryTitle    25107 non-null  object 
 2   startYear       25107 non-null  int64  
 3   runtimeMinutes  25107 non-null  int64  
 4   genres          25107 non-null  object 
 5   averageRating   25107 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 1.3+ MB


In [238]:
t_with_ratings.head()

,tconst,primaryTitle,startYear,runtimeMinutes,genres,averageRating
0,tt9109794,Kolaigaran,2019,109,"Action,Crime,Drama",7.1
1,tt3548862,Face Down,2015,110,"Crime,Drama",6.3
2,tt9916170,The Rehearsal,2019,51,Drama,7.0
3,tt1679625,Missione di pace,2011,90,"Comedy,Drama",5.7
4,tt0451769,Imaginum,2005,82,"Animation,Family,Fantasy",4.3


In [248]:
len(t_with_ratings.loc[t_with_ratings.averageRating < 7.0])/len(t_with_ratings) # 71% has a rating below 7.0

0.7080495479348389

In [67]:
def unique_genres(column):
    genres_set = set()
    for entry in column:
        list = entry.split(',')
        for item in list:
            genres_set.add(item.strip())
    return genres_set

In [180]:
genres = unique_genres(t_with_ratings.genres)

In [214]:
len(genres)

27

In [70]:
print(list(genres))

['Action', 'Documentary', 'Sport', 'War', 'Music', 'Talk-Show', 'Adult', 'Mystery', 'Reality-TV', 'Animation', 'Musical', 'Comedy', 'Adventure', 'Western', 'Film-Noir', 'Family', 'Drama', 'Horror', 'Romance', 'Crime', 'Biography', 'Fantasy', 'Sci-Fi', 'News', 'History', 'Thriller', 'Game-Show']


In [215]:
t_with_ratings[t_with_ratings.genres.str.contains('Reality-TV')]

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,averageRating
9192,tt2236650,movie,Sheryl Crow: Live,2011,92,"Music,Reality-TV",7.3
16718,tt3655682,movie,Let's Be Evil,2016,82,"Horror,Reality-TV,Sci-Fi",4.3
17318,tt0374226,movie,Small Mall,2004,67,"Comedy,Documentary,Reality-TV",7.4
18706,tt11934970,movie,Manchester United vs Manchester City,2020,100,Reality-TV,7.8
19596,tt13469602,movie,Zach Attacks: Costa Rica Especial,2018,70,Reality-TV,8.4


In [216]:
t_with_ratings[t_with_ratings.genres.str.contains('Game-Show')]

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,averageRating
12866,tt16585662,movie,Walking Craft - O Filme,2019,77,"Adventure,Comedy,Game-Show",9.0


In [217]:
t_with_ratings[t_with_ratings.genres.str.contains('News')] # 67 rows

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,averageRating
110,tt4242392,movie,Life is Waiting: Referendum and Resistance in ...,2015,61,"Documentary,History,News",7.0
1084,tt9328986,movie,This Little Land of Mines,2019,64,"Documentary,Drama,News",8.3
1366,tt3271310,movie,Revolution of the Present,2015,79,"Documentary,History,News",7.0
2216,tt1633267,movie,The Very Worst Thing,2010,85,"Documentary,History,News",8.2
2548,tt3283906,movie,"The British Film Industry: Elitist, Deluded or...",2014,101,"Documentary,History,News",6.4
...,...,...,...,...,...,...,...
22389,tt1776887,movie,Death by Medicine,2011,117,"Documentary,News",6.3
23506,tt2574306,movie,A2-B-C: Hôshanô to kodomotachi,2013,71,"Documentary,News",8.5
23963,tt1743943,movie,Libertopia,2010,87,"Documentary,News",7.2
24120,tt3164056,movie,Brothers of the Black List,2014,74,"Documentary,History,News",7.9


In [74]:
with open('genres_list.json', 'w') as f:
    json.dump(list(genres), f, indent=2)

In [246]:
_ = t_with_ratings.to_csv('titles_genres_ratings.csv', index=False)

# Netflix
data: https://www.kaggle.com/datasets/shivamb/netflix-shows \
columns: show_id, type, title, director, cast, country, date_added, release_year, rating, duration, listed_in, description

In [107]:
netflix = 'netflix_titles.csv'

In [183]:
nf = pd.read_csv(netflix, usecols=['show_id', 'type', 'title', 'release_year', 'rating', 'duration', 'listed_in'])

In [138]:
nf.head()

,show_id,title,release_year,rating,duration,listed_in
0,s1,Dick Johnson Is Dead,2020,PG-13,90,Documentaries
6,s7,My Little Pony: A New Generation,2021,PG,91,Children & Family Movies
7,s8,Sankofa,1993,TV-MA,125,"Dramas, Independent Movies, International Movies"
9,s10,The Starling,2021,PG-13,104,"Comedies, Dramas"
12,s13,Je Suis Karl,2021,TV-MA,127,"Dramas, International Movies"


In [222]:
nf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6131 entries, 0 to 8806
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6131 non-null   object
 1   title         6131 non-null   object
 2   release_year  6131 non-null   int64 
 3   rating        6129 non-null   object
 4   duration      6131 non-null   int64 
 5   listed_in     6131 non-null   object
dtypes: int64(2), object(4)
memory usage: 464.3+ KB


In [223]:
nf.describe()

,release_year,duration
count,6131.000000,6131.000000
mean,2013.121514,99.564998
std,9.678169,28.289504
min,1942.000000,3.000000
25%,2012.000000,87.000000
50%,2016.000000,98.000000
75%,2018.000000,114.000000
max,2021.000000,312.000000


In [185]:
# drop TV Show
nf.drop(nf[nf.type == 'TV Show'].index, inplace=True)

In [187]:
nf.drop(['type'], axis=1, inplace=True)

KeyError: "['type'] not found in axis"

In [218]:
listed_in = unique_genres(nf.listed_in)

In [219]:
len(listed_in)

20

In [220]:
print(listed_in)

{'Dramas', 'Children & Family Movies', 'LGBTQ Movies', 'Thrillers', 'Independent Movies', 'Sports Movies', 'Faith & Spirituality', 'Classic Movies', 'Movies', 'Music & Musicals', 'Horror Movies', 'Documentaries', 'Cult Movies', 'Stand-Up Comedy', 'Comedies', 'Anime Features', 'Romantic Movies', 'Sci-Fi & Fantasy', 'International Movies', 'Action & Adventure'}


In [85]:
with open('netflix_listed_in.json', 'w') as f:
    json.dump(list(listed_in), f, indent=2)

In [221]:
nf.release_year.min()

1942

In [188]:
# restore misplaced duration values for show_id= 's5542', 's5795', 's5814'
nf.loc[nf.duration.isnull(), ['duration']] = nf.rating

In [189]:
nf['duration'] = nf.duration.str.split(' ', expand=True)[0].astype('int64')

In [227]:
nf.loc[nf.rating.isnull()]

,show_id,title,release_year,rating,duration,listed_in
5989,s5990,13TH: A Conversation with Oprah Winfrey & Ava ...,2017,NaN,37,Movies
7537,s7538,My Honor Was Loyalty,2015,NaN,115,Dramas


In [224]:
nf.loc[nf.duration < 50] # 277 rows

,show_id,title,release_year,rating,duration,listed_in
45,s46,My Heroes Were Cowboys,2021,PG,23,Documentaries
71,s72,A StoryBots Space Adventure,2021,TV-Y,13,Children & Family Movies
77,s78,Little Singham - Black Shadow,2021,TV-Y7,48,"Children & Family Movies, Comedies"
470,s471,Bridgerton - The Afterparty,2021,TV-14,39,Movies
484,s485,Lee Su-geun: The Sense Coach,2021,TV-G,46,Stand-Up Comedy
...,...,...,...,...,...,...
8587,s8588,Thunderbolt,1947,TV-PG,42,"Classic Movies, Documentaries"
8657,s8658,Under an Arctic Sky,2017,TV-14,40,"Action & Adventure, Documentaries, Sports Movies"
8707,s8708,"We, the Marines",2017,TV-PG,38,"Documentaries, International Movies"
8759,s8760,World's Weirdest Homes,2015,TV-PG,49,Movies


# Netflix & IMDB

In [239]:
nf_imdb = nf.merge(t_with_ratings, 
                   how='inner', 
                   left_on=['title', 'release_year'], 
                   right_on=['primaryTitle', 'startYear'])

In [240]:
nf_imdb.head()

,show_id,title,release_year,rating,duration,listed_in,tconst,primaryTitle,startYear,runtimeMinutes,genres,averageRating
0,s36,The Father Who Moves Mountains,2021,TV-MA,110,"Dramas, International Movies, Thrillers",tt8886670,The Father Who Moves Mountains,2021,108,"Drama,Thriller",5.6
1,s89,Blood Brothers: Malcolm X & Muhammad Ali,2021,PG-13,96,"Documentaries, Sports Movies",tt15095938,Blood Brothers: Malcolm X & Muhammad Ali,2021,95,"Biography,Documentary,History",6.9
2,s92,The Women and the Murderer,2021,TV-14,92,"Documentaries, International Movies",tt15216504,The Women and the Murderer,2021,92,"Crime,Documentary",6.4
3,s139,Dear John,2010,PG-13,108,"Dramas, Romantic Movies",tt0989757,Dear John,2010,108,"Drama,Mystery,Romance",6.3
4,s146,House Party 2,1991,R,94,"Comedies, Cult Movies, Music & Musicals",tt0102065,House Party 2,1991,94,"Comedy,Drama,Music",5.2


In [226]:
nf_imdb.info() # 457 with imdb rating & genres out of 6131 titles in netflix data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457 entries, 0 to 456
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   show_id         457 non-null    object 
 1   title           457 non-null    object 
 2   release_year    457 non-null    int64  
 3   rating          457 non-null    object 
 4   duration        457 non-null    int64  
 5   listed_in       457 non-null    object 
 6   tconst          457 non-null    object 
 7   titleType       457 non-null    object 
 8   primaryTitle    457 non-null    object 
 9   startYear       457 non-null    int64  
 10  runtimeMinutes  457 non-null    int64  
 11  genres          457 non-null    object 
 12  averageRating   457 non-null    float64
dtypes: float64(1), int64(4), object(8)
memory usage: 50.0+ KB


In [233]:
len(nf_imdb.loc[abs(nf_imdb.duration - nf_imdb.runtimeMinutes > 10)]) # all refer to the same movies

13

In [242]:
nf_imdb.drop(['show_id', 'primaryTitle', 'startYear', 'runtimeMinutes'], axis=1, inplace=True)

In [245]:
len(nf_imdb.loc[nf_imdb.averageRating < 7.0])/len(nf_imdb) # 77% has a rating below 7.0

0.7702407002188184

In [247]:
_ = nf_imdb.to_csv('netflix_imdb.csv', index=False)

In [250]:
nf_imdb.loc[nf_imdb.averageRating > 7.0]

,title,release_year,rating,duration,listed_in,tconst,genres,averageRating
8,Once Upon a Time in America,1984,R,229,"Classic Movies, Dramas",tt0087843,"Crime,Drama",8.3
13,Bombay,1995,TV-14,135,"Dramas, International Movies, Romantic Movies",tt0112553,"Drama,Romance",8.1
14,Untold: Malice at the Palace,2021,TV-MA,70,"Documentaries, Sports Movies",tt15085802,"Crime,Documentary,Sport",7.5
18,Fantastic Fungi,2019,TV-14,81,Documentaries,tt8258074,Documentary,7.4
24,Seven Pounds,2008,PG-13,123,"Dramas, Romantic Movies",tt0814314,Drama,7.6
...,...,...,...,...,...,...,...,...
435,The Phantom of the Opera,2004,PG-13,141,"Dramas, Music & Musicals, Romantic Movies",tt0293508,"Drama,Musical,Romance",7.2
444,Trophy,2017,TV-MA,110,"Documentaries, International Movies",tt6333066,Documentary,7.4
447,Tyke Elephant Outlaw,2015,TV-14,78,"Documentaries, International Movies",tt4254838,Documentary,7.4
449,Waiting,2015,TV-MA,94,"Dramas, International Movies",tt3531740,"Documentary,Drama",7.7


In [254]:
nf_imdb.sort_values('averageRating', ascending=False)[:25]

,title,release_year,rating,duration,listed_in,tconst,genres,averageRating
357,Krish Trish and Baltiboy: Face Your Fears,2017,TV-Y7,65,Children & Family Movies,tt8672310,Animation,9.0
84,Listen,2017,TV-MA,103,"Dramas, Independent Movies, International Movies",tt3766102,Drama,9.0
360,Life in the Doghouse,2018,TV-PG,83,"Documentaries, LGBTQ Movies",tt5178264,Documentary,8.4
391,Sairat,2016,TV-14,173,"Dramas, International Movies, Romantic Movies",tt5312232,"Drama,Romance",8.3
8,Once Upon a Time in America,1984,R,229,"Classic Movies, Dramas",tt0087843,"Crime,Drama",8.3
191,Andhadhun,2018,TV-14,137,"Comedies, Dramas, International Movies",tt8108198,"Comedy,Crime,Music",8.2
130,Miracle in Cell No. 7,2019,TV-14,132,"Dramas, International Movies",tt10431500,Drama,8.2
386,Raging Bull,1980,R,129,"Classic Movies, Dramas, Sports Movies",tt0081398,"Biography,Drama,Sport",8.2
147,Klaus,2019,PG,98,"Children & Family Movies, Comedies",tt4729430,"Adventure,Animation,Comedy",8.1
174,Shonar Pahar,2018,TV-PG,131,"Dramas, Independent Movies, International Movies",tt8347882,Drama,8.1
